# 1. Famílias inscritas no Cadastro Único por faixa de renda per capita

# 2. Membros
Thiago Augusto Vaz Silveira - 2023001506

Vanessa Nascimento Silva - 2023001816

Yuri Estevao Sales de Oliveira - 2023087850


# 3. Descrição dos dados (qual a URL? qual o domínio? como os dados foram processados?)

In [ ]:
import sqlite3
import pandas as pd
import requests

# URL do arquivo no GitHub
url = "https://github.com/ncsvanessa/TP2_IBD/raw/refs/heads/main/dados.db"

# Nome do arquivo para salvar localmente
db_path = "dados.db"

# Baixar o arquivo
response = requests.get(url)
if response.status_code == 200:
    with open(db_path, 'wb') as file:
        file.write(response.content)
    print(f"Banco de dados baixado com sucesso em: {db_path}")
else:
    print(f"Erro ao baixar o banco de dados: {response.status_code}")

# Conexão com o banco de dados
conn = sqlite3.connect("dados.db")
cursor = conn.cursor()



# 4. Diagrama ER

![alt text](diagrama_er.drawio.png)

In [ ]:
from IPython.display import Image, display

# URL da imagem
url = "https://i.postimg.cc/CLnNtmv1/diagrama-er-drawio.png"

# Exibir a imagem
display(Image(url=url))


# 5. Diagrama relacional

![alt text](diagrama_relacional.drawio.png)

In [ ]:
from IPython.display import Image, display

# URL da imagem
url = "https://i.postimg.cc/Gmkj39nn/diagrama-relacional-drawio.png"

# Exibir a imagem
display(Image(url=url))

# 6. Consultas

Teste 1: Listar as tabelas no banco

In [ ]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
print("Tabelas disponíveis:")
print(tables)

Teste 2: Conferir as junções de tabela

In [ ]:
query = """
SELECT Municipio.nome_municipio, FamiliasPobres.qtd_familias
FROM Municipio
JOIN FamiliasPobres  
    ON CAST(TRIM(Municipio.codigo_ibge) AS TEXT) = CAST(TRIM(FamiliasPobres.codigo_ibge) AS TEXT)
WHERE TRIM(Municipio.uf) = 'SP' AND FamiliasPobres.mes = '202401'
ORDER BY FamiliasPobres.qtd_familias DESC
LIMIT 10;
"""

df = pd.read_sql_query(query, conn)
df

## 6.1 Duas consultas envolvendo seleção e projeção

### 6.1.1 Consulta 1

Municípios do estado de São Paulo com mais de 10.000 famílias em situação de pobreza, segundo a faixa do Programa Bolsa Família, em janeiro de 2024

In [ ]:
query = """
SELECT 
    Municipio.nome_municipio,
    FamiliasPobres.qtd_familias
FROM Municipio
JOIN FamiliasPobres 
    ON CAST(TRIM(Municipio.codigo_ibge) AS TEXT) = CAST(TRIM(FamiliasPobres.codigo_ibge) AS TEXT)
WHERE TRIM(Municipio.uf) = 'SP' AND FamiliasPobres.qtd_familias > 10000 AND FamiliasPobres.mes = '202401'
ORDER BY FamiliasPobres.qtd_familias DESC
"""

df = pd.read_sql_query(query, conn)
df

### 6.1.2 Consulta 2 // muito simples

Municípios do estado de Rondônia

In [ ]:
query = """
SELECT 
    Municipio.nome_municipio, 
    Municipio.uf
FROM Municipio
WHERE TRIM(Municipio.uf) = 'RO'
ORDER BY Municipio.nome_municipio;
"""

df = pd.read_sql_query(query, conn)
df

## 6.2 Três consultas envolvendo junção de duas relações

### 6.2.1 Consulta 3

Os 10 municípios com o maior indicador de famílias em situação de pobreza, segundo a faixa do Programa Bolsa Família, em janeiro de 2024

In [ ]:
query = """
SELECT 
    Municipio.nome_municipio,
    FamiliasPobres.qtd_familias AS familias_pobres
FROM Municipio
JOIN FamiliasPobres 
    ON Municipio.codigo_ibge = FamiliasPobres.codigo_ibge
WHERE FamiliasPobres.mes = '202401'
ORDER BY FamiliasPobres.qtd_familias DESC
LIMIT 10;
"""

df = pd.read_sql_query(query, conn)
df

### 6.2.2 Consulta 4 // parecida com a consulta 8

Total de famílias em situação de pobreza, segundo a faixa do Programa Bolsa Família, por município em Rondônia no período de janeiro de 2024

In [ ]:
query = """
SELECT 
    Municipio.nome_municipio, 
    SUM(FamiliasPobres.qtd_familias) AS total_familias_pobres
FROM Municipio
JOIN FamiliasPobres
    ON Municipio.codigo_ibge = FamiliasPobres.codigo_ibge
WHERE TRIM(Municipio.uf) = 'RO' AND FamiliasPobres.mes = '202401'
GROUP BY Municipio.nome_municipio
ORDER BY Municipio.nome_municipio;
"""

df = pd.read_sql_query(query, conn)
df

### 6.2.3 Consulta 5 //mesma pesquisa que a 9

Total de famílias em situação de pobreza, segundo a faixa do Programa Bolsa Família, por estado (UF) no período de janeiro de 2024

In [ ]:
query = """
SELECT 
    TRIM(Municipio.uf) AS uf, 
    SUM(FamiliasPobres.qtd_familias) AS total_familias_pobres
FROM Municipio
JOIN FamiliasPobres
    ON Municipio.codigo_ibge = FamiliasPobres.codigo_ibge
WHERE FamiliasPobres.mes = '202401'
GROUP BY TRIM(Municipio.uf)
ORDER BY TRIM(Municipio.uf);
"""

df = pd.read_sql_query(query, conn)
df

## 6.3 Três consultas envolvendo junção de três ou mais relações

### 6.3.1 Consulta 6

Quantidade de famílias por indicador listadas por período no país no ano de 2024 

In [ ]:
query = """
SELECT 
    FamiliasPobres.mes AS periodo,
    SUM(FamiliasPobres.qtd_familias) AS total_familias_pobres,
    SUM(FamiliasBaixaRenda.qtd_familias) AS total_familias_baixa_renda,
    SUM(FamiliasAcimaMeioSalario.qtd_familias) AS total_familias_acima_meio_salario
FROM FamiliasPobres
LEFT JOIN FamiliasBaixaRenda 
    ON FamiliasPobres.codigo_ibge = FamiliasBaixaRenda.codigo_ibge AND FamiliasPobres.mes = FamiliasBaixaRenda.mes
LEFT JOIN FamiliasAcimaMeioSalario 
    ON FamiliasPobres.codigo_ibge = FamiliasAcimaMeioSalario.codigo_ibge AND FamiliasPobres.mes = FamiliasAcimaMeioSalario.mes
GROUP BY FamiliasPobres.mes
ORDER BY FamiliasPobres.mes;
"""

df = pd.read_sql_query(query, conn)
df

### 6.3.2 Consulta 7

Município de cada estado(UF) com os maiores valores por indicador no período de janeiro de 2024


In [ ]:
query = """
WITH RankedCities AS (
    SELECT 
        Municipio.uf,
        Municipio.nome_municipio,
        FamiliasPobres.qtd_familias AS familias_pobres,
        FamiliasBaixaRenda.qtd_familias AS familias_baixa_renda,
        FamiliasAcimaMeioSalario.qtd_familias AS familias_acima_meio_salario,
        ROW_NUMBER() OVER (PARTITION BY Municipio.uf ORDER BY FamiliasPobres.qtd_familias DESC) AS rank_pobres,
        ROW_NUMBER() OVER (PARTITION BY Municipio.uf ORDER BY FamiliasBaixaRenda.qtd_familias DESC) AS rank_baixa_renda,
        ROW_NUMBER() OVER (PARTITION BY Municipio.uf ORDER BY FamiliasAcimaMeioSalario.qtd_familias DESC) AS rank_acima_meio_salario
    FROM Municipio
    LEFT JOIN FamiliasPobres 
        ON Municipio.codigo_ibge = FamiliasPobres.codigo_ibge AND FamiliasPobres.mes = '202401'
    LEFT JOIN FamiliasBaixaRenda 
        ON Municipio.codigo_ibge = FamiliasBaixaRenda.codigo_ibge AND FamiliasBaixaRenda.mes = '202401'
    LEFT JOIN FamiliasAcimaMeioSalario 
        ON Municipio.codigo_ibge = FamiliasAcimaMeioSalario.codigo_ibge AND FamiliasAcimaMeioSalario.mes = '202401'
)
SELECT 
    uf,
    nome_municipio,
    familias_pobres,
    familias_baixa_renda,
    familias_acima_meio_salario
FROM RankedCities
WHERE rank_pobres = 1 OR rank_baixa_renda = 1 OR rank_acima_meio_salario = 1
ORDER BY uf;
"""

df = pd.read_sql_query(query, conn)
df

### 6.3.3 Consulta 8 // parecida com a consulta 4

Municípios de Rondônia por indicador no período de janeiro de 2024

In [ ]:
query = """
SELECT 
    Municipio.nome_municipio, 
    SUM(FamiliasPobres.qtd_familias) AS familias_pobres, 
    SUM(FamiliasBaixaRenda.qtd_familias) AS familias_baixa_renda, 
    SUM(FamiliasAcimaMeioSalario.qtd_familias) AS familias_acima_meio_salario
FROM Municipio
LEFT JOIN FamiliasPobres
    ON Municipio.codigo_ibge = FamiliasPobres.codigo_ibge AND FamiliasPobres.mes = '202401'
LEFT JOIN FamiliasBaixaRenda
    ON Municipio.codigo_ibge = FamiliasBaixaRenda.codigo_ibge AND FamiliasBaixaRenda.mes = '202401'
LEFT JOIN FamiliasAcimaMeioSalario
    ON Municipio.codigo_ibge = FamiliasAcimaMeioSalario.codigo_ibge AND FamiliasAcimaMeioSalario.mes = '202401'
WHERE TRIM(Municipio.uf) = 'RO'
GROUP BY Municipio.nome_municipio
ORDER BY Municipio.nome_municipio;
"""

df = pd.read_sql_query(query, conn)
df

## 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

### 6.4.1 Consulta 9

Soma total de famílias por estado (UF) para cada indicador no período de janeiro de 2024

In [ ]:
query = """
SELECT 
    Municipio.uf,
    SUM(FamiliasPobres.qtd_familias) AS total_familias_pobres,
    SUM(FamiliasBaixaRenda.qtd_familias) AS total_familias_baixa_renda,
    SUM(FamiliasAcimaMeioSalario.qtd_familias) AS total_familias_acima_meio_salario
FROM Municipio
LEFT JOIN FamiliasPobres 
    ON Municipio.codigo_ibge = FamiliasPobres.codigo_ibge AND FamiliasPobres.mes = '202401' 
LEFT JOIN FamiliasBaixaRenda 
    ON Municipio.codigo_ibge = FamiliasBaixaRenda.codigo_ibge AND FamiliasBaixaRenda.mes = '202401' 
LEFT JOIN FamiliasAcimaMeioSalario 
    ON Municipio.codigo_ibge = FamiliasAcimaMeioSalario.codigo_ibge AND FamiliasAcimaMeioSalario.mes = '202401' 
GROUP BY Municipio.uf
ORDER BY Municipio.uf;
"""

df = pd.read_sql_query(query, conn)
df

### 6.4.2 Consulta 10 

Média de famílias em situação de pobreza, segundo a faixa do Programa Bolsa Família, por estado (UF) no período de janeiro de 2024

In [ ]:
query = """
SELECT 
    TRIM(Municipio.uf) AS uf, 
    AVG(FamiliasPobres.qtd_familias) AS media_familias_pobres
FROM Municipio
JOIN FamiliasPobres
    ON Municipio.codigo_ibge = FamiliasPobres.codigo_ibge
WHERE FamiliasPobres.mes = '202401'
GROUP BY TRIM(Municipio.uf)
ORDER BY TRIM(Municipio.uf);
"""

df = pd.read_sql_query(query, conn)
df

# 7. Autoavaliação dos membros